In [2]:
import pandas as pd
import sqlite3
import seaborn as sns
import matplotlib.pyplot as plt

In [6]:
def main():
    conn = sqlite3.connect("C:/Users/megan.lieu/Downloads/SQLCaseStudy/SQLCaseStudy/CaseStudy.db")
   
    #tables for Marvel questions
    
    
#     WORK FOR #1 - MARVEL
    sql1 = "select name, CAST(Num_Appear AS INT) AS Num_Appearances from (\
    select name, APPEARANCES, ROW_NUMBER() OVER (ORDER BY CAST(APPEARANCES AS INT)) AS Top10Appearances\
    , ROW_NUMBER() OVER (ORDER BY CAST(APPEARANCES AS INT) desc) AS Bottom10Appearances\
    , CASE WHEN APPEARANCES = '' THEN 0 ELSE APPEARANCES END AS 'Num_Appear'\
    from Marvel_Character\
    WHERE APPEARANCES != '') \
    WHERE Top10Appearances <= 10 or Bottom10Appearances <= 10;"

    df_MC = pd.read_sql_query(sql1, conn)
    print(df_MC)
    
    
#     sql = """select name, CAST(APPEARANCES AS INT) AS Bottom_Num_Appearances from Marvel_Character
#     union
#     select name, CAST(APPEARANCES AS INT) AS Top_Num_Appearances from Marvel_Character
#     order by Bottom_Num_Appearances, Top_Num_Appearances desc limit 10"""


#     WORK FOR #2 - MARVEL
    sql2 = "SELECT name, FIRST_APPEARANCE, year_first_appear, Num_Years_Appeared FROM (\
    SELECT name, FIRST_APPEARANCE, year_first_appear, ROW_NUMBER() OVER (ORDER BY 2022 - CAST(year_first_appear AS INT) DESC) AS Top10TimeAppear\
    , ROW_NUMBER() OVER (ORDER BY 2022 - CAST(year_first_appear AS INT)) AS Bottom10TimeAppear\
    , CASE WHEN year_first_appear = '' THEN 0 ELSE 2022 - CAST(year_first_appear AS INT) END AS 'Num_Years_Appeared'\
    FROM Marvel_Character \
    WHERE FIRST_APPEARANCE != '')\
    WHERE Top10TimeAppear <= 10 or Bottom10TimeAppear <= 10;"

#     sql = "select substr('JanFebMarAprMayJunJulAugSepOctNovDec', strftime('%m', FIRST_APPEARANCE) * 3, 3) from Marvel_Character"
    df_MC = pd.read_sql_query(sql2, conn)
    print(df_MC)
    
#     WORK FOR #3 - MARVEL    
    sql3 = "WITH GenderCount AS (\
    SELECT a.name, a.year_first_appear, b.SEX\
    FROM Marvel_Character a INNER JOIN Marvel_Description b\
    ON a.page_id = b.page_id)\
    , DecadeCount AS (\
    SELECT name, SEX, CASE \
        WHEN year_first_appear LIKE '193%' THEN '1930s'\
        WHEN year_first_appear LIKE '194%' THEN '1940s'\
        WHEN year_first_appear LIKE '195%' THEN '1950s'\
        WHEN year_first_appear LIKE '196%' THEN '1960s'\
        WHEN year_first_appear LIKE '197%' THEN '1970s'\
        WHEN year_first_appear LIKE '198%' THEN '1980s'\
        WHEN year_first_appear LIKE '199%' THEN '1990s'\
        WHEN year_first_appear LIKE '200%' THEN '2000s'\
        WHEN year_first_appear LIKE '201%' THEN '2010s'\
        ELSE '2020s' END AS decade_first_appear\
    from GenderCount)\
    \
    SELECT decade_first_appear AS Decade, \
    CASE WHEN SEX = '' THEN 'Unspecified' ELSE SEX END AS Gender, \
    COUNT(*) AS Count, \
    (COUNT(*) * 100.0 / SUM(COUNT(*)) OVER (PARTITION BY decade_first_appear)) AS 'GenderRatio (%)'\
    FROM DecadeCount\
    GROUP BY decade_first_appear, SEX"

    df_MD = pd.read_sql_query(sql3, conn)
    print(df_MD)


#   WORK FOR #4

    sql4 = "SELECT distinct b.SEX, a.APPEARANCES, COUNT(*) AS Frequency\
    FROM Marvel_Character a\
    left join Marvel_Description b\
    ON a.page_id = b.page_id\
    WHERE b.SEX != ''\
    AND a.APPEARANCES != ''\
    GROUP BY b.SEX, a.APPEARANCES\
    ORDER BY COUNT(*) DESC"
    
#     sql4 = "WITH \
#         Appearances_by_Eyes AS (\
#         SELECT a.name, a.year_first_appear, a.APPEARANCES, b.EYE, b.SEX\
#         FROM Marvel_Character a INNER JOIN Marvel_Description b\
#         on a.page_id = b.page_id),\
#         \
#         Appearances_Counts AS (\
#         SELECT CASE WHEN SEX = '' THEN 'Unspecified Gender' ELSE SEX END AS Gender\
#         , CASE WHEN EYE = '' THEN 'Unspecified Eyes' ELSE EYE END AS Eye_Color\
#         , SEX || ' + ' || EYE AS Category\
#         , SUM(APPEARANCES) AS Num_Appearances\
#         FROM Appearances_by_Eyes\
#         GROUP BY SEX, EYE\
#         ORDER BY SUM(APPEARANCES) DESC)\
#         \
#         SELECT * FROM Appearances_by_Eyes WHERE EYE LIKE '%brown%' AND SEX LIKE '%male%'"

#     [FOR SUMMARY STATISTICS FOR BROWN-EYED CHARACTERS]:      
#         SELECT SEX, EYE, COUNT(*) AS 'Count', SUM(CAST(APPEARANCES AS INT)) AS Total_Appearances, AVG(CAST(APPEARANCES AS INT)) AS Average_Appearances, MAX(CAST(APPEARANCES AS INT)) AS Max_Appearances, MIN(CAST(APPEARANCES AS INT)) AS Min_Appearances\
#         FROM Appearances_by_Eyes\
#         WHERE EYE LIKE '%brown%'\
#         AND SEX LIKE '%male%'\
#         AND SEX NOT LIKE '%female%'\
#         AND APPEARANCES != ''\
#         GROUP BY SEX, EYE\
#         UNION\
#         SELECT SEX, EYE, COUNT(*) AS 'Count', SUM(CAST(APPEARANCES AS INT)) AS Total_Appearances, AVG(CAST(APPEARANCES AS INT)) AS Average_Appearances, MAX(CAST(APPEARANCES AS INT)) AS Max_Appearances, MIN(CAST(APPEARANCES AS INT)) AS Min_Appearances\
#         FROM Appearances_by_Eyes\
#         WHERE EYE LIKE '%brown%'\
#         AND SEX LIKE '%female%'\
#         AND APPEARANCES != ''\
#         GROUP BY SEX, EYE"
    
#     [FOR FULL DISTRIBUTION OF ALL EYE COLORS AND GENDERS]:    
#     SELECT Gender, Eye_Color, Gender || ' + ' || Eye_Color AS Description, Num_Appearances FROM Appearances_Counts\
#     WHERE Eye_Color LIKE '%brown%'\
#     AND (Gender LIKE '%male%' OR Gender LIKE '%female%')"

#     [FOR STATS ON ENTIRE POPULATION BY GENDER AND EYE COLOR]:
#     SELECT a.Category, a.Num_Appearances\
#     FROM Appearances_Counts a\
#     INNER JOIN (\
#         SELECT Category\
#         , MAX(Num_Appearances) Num_Appearances\
#         FROM Appearances_Counts\
#     ) b ON a.Category = b.Category AND a.Num_Appearances = b.Num_Appearances\
#     UNION\
#     SELECT a.Category, a.Num_Appearances\
#     FROM Appearances_Counts a\
#     INNER JOIN (\
#         SELECT Category\
#         , MIN(Num_Appearances) Num_Appearances\
#         FROM Appearances_Counts\
#     ) b ON a.Category = b.Category AND a.Num_Appearances = b.Num_Appearances"

    
    df_MD = pd.read_sql_query(sql4, conn)
    print(df_MD.to_clipboard())

#   MORE DISTRIBUTION STATS    
    print(df_MD['APPEARANCES'].describe())
    
#     fig, ax = plt.subplots(figsize = ( 20 , 15 ))
#     sns.histplot(df_MD['APPEARANCES'])
#     ax.set_xlabel('Appearances')
#     plt.ylabel("Count")
#     plt.xticks(rotation=60, fontsize=7.5)
#     plt.grid()
#     plt.show()
        
if __name__=="__main__":
    main()

                                     name  Num_Appearances
0                      Bardak (Earth-616)                1
1       Bends (Masked Raider) (Earth-616)                1
2                Blackie Ross (Earth-616)                1
3                       Bleck (Earth-616)                1
4                 Cal Brunder (Earth-616)                1
5              Constance Rand (Earth-616)                1
6   Dead Shot (Masked Raider) (Earth-616)                1
7     Dr. Lang (The Big Boss) (Earth-616)                1
8                Dutch Hansen (Earth-616)                1
9                       Ganya (Earth-616)                1
10             Jonathan Storm (Earth-616)             1934
11              Scott Summers (Earth-616)             1955
12             Hulk (Robert Bruce Banner)             2017
13              Reed Richards (Earth-616)             2072
14             Benjamin Grimm (Earth-616)             2255
15                    Thor (Thor Odinson)             22

In [9]:
def main():
    conn = sqlite3.connect("C:/Users/megan.lieu/Downloads/SQLCaseStudy/SQLCaseStudy/CaseStudy.db")
   
    #tables for DC questions
    
    
#     WORK FOR #1 - DC    
    sql1 = "SELECT name, CAST(Num_Appear AS INT) AS Num_Appearances FROM (\
     SELECT name, APPEARANCES, ROW_NUMBER() OVER (ORDER BY CAST(APPEARANCES AS INT)) AS Top10Appearances\
     , ROW_NUMBER() OVER (ORDER BY CAST(APPEARANCES AS INT) desc) AS Bottom10Appearances\
     , CASE WHEN APPEARANCES = '' THEN 0 ELSE APPEARANCES END AS 'Num_Appear'\
     FROM DC_Character\
     WHERE APPEARANCES != '') \
     WHERE Top10Appearances <= 10 or Bottom10Appearances <= 10;"
    
    df_MC = pd.read_sql_query(sql1, conn)
    print(df_MC)    
    

#     WORK FOR #2 - DC
    sql2 = "select * from DC_Character"
    
#     "SELECT name, FIRST_APPEARANCE, year_first_appear, Num_Years_Appeared FROM (\
#     SELECT name, FIRST_APPEARANCE, year_first_appear, ROW_NUMBER() OVER (ORDER BY 2022 - CAST(year_first_appear AS INT) DESC) AS Top10TimeAppear\
#     , ROW_NUMBER() OVER (ORDER BY 2022 - CAST(year_first_appear AS INT)) AS Bottom10TimeAppear\
#     , CASE WHEN year_first_appear = '' THEN 0 ELSE 2022 - CAST(year_first_appear AS INT) END AS 'Num_Years_Appeared'\
#     FROM DC_Character \
#     WHERE FIRST_APPEARANCE != '')\
#     WHERE Top10TimeAppear <= 10 or Bottom10TimeAppear <= 10;"


    df_MC = pd.read_sql_query(sql2, conn)
    print(df_MC)
    
#     WORK FOR #3 - DC
    sql3 = "select a.*, b.ALIGN, b.EYE, b.HAIR, b.SEX, b.GSM, b.ALIVE\
    from DC_Character a\
    left join DC_Description b\
    ON a.page_id = b.page_id"
    
#     "WITH GenderCount AS (\
#     SELECT a.name, a.year_first_appear, b.SEX\
#     FROM DC_Character a INNER JOIN DC_Description b\
#     ON a.page_id = b.page_id)\
#     , DecadeCount AS (\
#     SELECT name, SEX, CASE \
#         WHEN year_first_appear LIKE '193%' THEN '1930s'\
#         WHEN year_first_appear LIKE '194%' THEN '1940s'\
#         WHEN year_first_appear LIKE '195%' THEN '1950s'\
#         WHEN year_first_appear LIKE '196%' THEN '1960s'\
#         WHEN year_first_appear LIKE '197%' THEN '1970s'\
#         WHEN year_first_appear LIKE '198%' THEN '1980s'\
#         WHEN year_first_appear LIKE '199%' THEN '1990s'\
#         WHEN year_first_appear LIKE '200%' THEN '2000s'\
#         WHEN year_first_appear LIKE '201%' THEN '2010s'\
#         ELSE '2020s' END AS decade_first_appear\
#     from GenderCount)\
#     \
#     SELECT decade_first_appear AS Decade, \
#     CASE WHEN SEX = '' THEN 'Unspecified' ELSE SEX END AS Gender, \
#     COUNT(*) AS Count, \
#     (COUNT(*) * 100.0 / SUM(COUNT(*)) OVER (PARTITION BY decade_first_appear)) AS 'GenderRatio (%)'\
#     FROM DecadeCount\
#     GROUP BY decade_first_appear, SEX"    
    
    
    df_MD = pd.read_sql_query(sql3, conn)
    print(df_MD.to_clipboard())
    
if __name__=="__main__":
    main()

                               name  Num_Appearances
0   Springheeled Jack (Prime Earth)                1
1              Napalm (Prime Earth)                1
2               Adellca (New Earth)                1
3                Armory (New Earth)                1
4                Artois (New Earth)                1
5                   Aya (New Earth)                1
6             Backslash (New Earth)                1
7            Bayfrentos (New Earth)                1
8             Big Mummy (New Earth)                1
9   Bizarro Blue Beetle (New Earth)                1
10              Flash (Barry Allen)             1028
11   Dinah Laurel Lance (New Earth)             1075
12        Timothy Drake (New Earth)             1095
13           Aquaman (Arthur Curry)             1121
14      Wonder Woman (Diana Prince)             1231
15      Richard Grayson (New Earth)             1237
16         James Gordon (New Earth)             1316
17       Green Lantern (Hal Jordan)           